# The Deep Learning Book (Simplified)
## Part II - Modern Practical Deep Networks
*This is a series of blog posts on the [Deep Learning book](http://deeplearningbook.org)
where we are attempting to provide a summary of each chapter highlighting the concepts 
that we found to be most important so that other people can use it as a starting point
for reading the chapters, while including the code for reproducing some of the results. 
Please refer [this](http://www.deeplearningbook.org/contents/notation.html) for more clarity on 
notation.*


## Chapter 8: Optimization for Training Deep Models

There are many types of optimization problems involved in deep learning, with the toughest one being that of neural network training. The main theme of the chapter is to focus on one type of optimization - finding the parameters $\theta$ that reduces a cost function $J(\theta)$. <br>

The chapter is organized as follows:

**1. How Learning Differs from Pure Optimization** <br>
**2. Challenges in Neural Network Optimization** <br>
**3. Basic Algorithms** <br>
**4. Parameter Initialization Strategies** <br>
**5. Algorithms with Adaptive Learning Rates** <br>
**6. Approximate Second-Order Methods** <br>
**7. Optimization Strategies and Meta-Algorithms** 

### 1. How Learning differs from Pure Optimization

- In Machine Learning (ML), we care about a certain performance measure *P* defined w.r.t the test set and optimize *$J(\theta)$* with the hope that it improves *P* as well. In pure optimization, optimizing *$J(\theta)$* is the final goal.

- The expected generalization error (**risk**) is taken over the true data-generating distribution $p_{data}$. If we do have that, it becomes an optimization problem. When we don't have $p_{data}$ but a finite training set, we have a ML problem. The latter can be converted back to an optimization problem by replace $p_{data}$ with the empirical distribution, $\tilde{p}_{data}$ obtained from the training set, thereby reducing the empirical risk. This is called empirical risk minimization (ERM) and it might look relatively similar to optimization. But there are two main problems:
    - ERM is prone to overfitting with the possibility of the dataset being learned by high capacity models.
    - ERM might not be feasible. Most optimization algorithms now are based on Gradient Descent (GD) which may not work with various loss functions like 0-1 loss (as it is not differentiable).
    
- It is for the reasons mentioned above that a **surrogate loss function** (SLF) is used instead that acts as a proxy. E.g. the negative log-likelihood of the true class is used as a surrogate for 0-1 loss. Using a SLF might even turn out to be beneficial as you can keep continuing to obtain a better test error by pushing the classes even further apart to get a more reliable classifier.

- Another common difference is that training might be halted following some convergence criterion based on Early Stopping to prevent overfitting, when the derivative of the surrogate loss function might still be large. This is different from pure optimization which is halted only when the derivative becomes very small.

- In ML optimization algorithms that objective function decomposes as a sum over the examples and we can perform updates by randomly sampling a batch of examples and taking the average over those examples. The Standard Error of the mean estimated from *n* examples is given by the $\frac {\sigma} {\sqrt{n}}$ indicating that as we include more examples for making an update, the returns of additonal examples in improving the error is less than linear. Thus, if we use 100 and 10000 example s to make an update, the latter takes 100 times more compute, but reducing the error only by a factor of 10. Thus, it's better to compute rapid approximate updates rather than a slow exact update.

- There are 3 types of sampling based algorithms - **batch gradient descent (BGD)** where the entire training set is used to make a single update, **stochastic gradient descent (SGD)** where a single example is used to make an update and **mini-batch gradient descent (MBGD)** where a batch (not to be confused with BGD) of examples is randomly sampled from the entire training set and is used to make an update. *MBGD is nowadays commonly refered to as SGD*. It is a common practise to use batch sizes as powers of 2 to offer better runtime with certain hardware. Small batches tend to have a regularizing effect because of the noise they inject due to the random sampling of the mini-batch.

- The minibatches should be selected randomly. It is sufficient to shuffle the dataset once and iterate over it multiple times. In the first epoch, the network sees each example for the first time and hence, the estimate of gradient is an unbiased estimate of the gradient of the true generalization error. However, from the second epoch onwards, the estimate becomes biased as it is resampling from data that it has already seen.



### 2. Challenges in Neural Network Optimization

The optimization problem for training neural networks is generally nonconvex. Some of the challenges faced are mentioned below:

- **Ill-conditioning of the Hessian Matrix**: Can cause the SGD to get stuck in a sense that even very small steps increase the cost function. Moving by a factor of $-\epsilon \mathbf{g}$  would add the term given below to the cost function. Ill-conditioning is to happen when the first term exceeds the second term. In many cases, $\mathbf{g}^TH\mathbf{g}$ does increase significantly leading to slower learning due to reduced learning rate. Thus, while the gradient norm might be increasing, it can still lead to successful training.

$$ \frac{1}{2}\epsilon^2\mathbf{g}^TH\mathbf{g} - \epsilon\mathbf{g}^T\mathbf{g} $$

- **Local minima**: Nearly any DL model is guaranteed to have an extremely large number of local minima (LM) arising due to the model identifiability problem. A model is said to be identifiable if a sufficiently large training set can rule out all but one setting of the model parameters. In case of neural networks, we can obtain equivalent models by swapping the position of the neurons. Thus, they are not identifiable. Thus, all the local minima caused due to this have the same value, thus not being a problem. However, if local minima with high cost are common, it becomes a serious problem. Many points other than local minima can lead to low gradients. Nowadays, it's common to aim for a low but not minimal cost value.

- **Plateaus, Saddle Points and Other Flat Regions**: Saddle point (SP) is another type of point with zero gradient where some points around it have higher value and the others have lower. Thus, Hessian at SP has both +ve and -ve eigenvalues. For many classes of random functions, saddle points become more common at high dimensions  with the ratio of number of SPs to LMs growing exponentially with *n* for an n-dimensional space. Many random functions have an amazing property that near points with lost cost, the Hessian tends to take up mostly positive eigenvalues. SGD empirically tends to rapidly avoid encountering a high-cost saddle point. There also might be wide, flat regions of constant value, thereby having a zero gradient. These can be problematic if the cost is high in these regions.

- **Cliffs and Exploding Gradients**: NNs might sometimes have extremely steep regions resembling cliffs due to the repeated multiplication of weights. At the edge of such a cliff, an update step might throw the parameters extremely far. It can be taken care of by using **gradient clipping (GC)**. The gradient indicates only the direction in which to make the update. If the GD update proposes making a very large step, GC intervenes to reduce the step size.
![gradient clipping](images/gradient_clipping.png)

- **Long-Term Dependencies**: This problem is encountered when the NN becomes sufficiently deep. For example, if the same weight matrix $\mathbf{W}$ is used in each layer, after *t* steps, we'd get $\mathbf{W}^t$. Using the eigendecomposition of $\mathbf{W} = \mathbf{V} diag(\lambda) \mathbf{V}^T$, $\mathbf{W}^t = \mathbf{V} diag(\lambda)^t \mathbf{V}^T$. Thus, any eigenvalues not near an absolute value of 1 would either explode or vanish leading to the **vanishing and exploding gradient problem**. The use of the same weight matrix is especially the case in RNNs where this is a serious problem.

- **Inexact Gradients**: Most optimization algorithms use a noisy/biased estimate of the gradient in cases where the estimate is based on sampling, while in cases where the true gradient is intractable, an approximation of the gradient is used. E.g. the contrastive divergence algorithm in the case of an RBM gives a technique for approximating the gradient of its intractable log-likelihood.

- Neural Networks might not end up at any critical point at all and such critical points might not even necessarily exist. A lot of the problems might be avoided if there exists a space connected to reasonably directly to the solution by a path that local descent can follow and if we are able to initialize learning within that well-behaved space. Thus, choosing good initial points should be studied.